In [1]:
from scipy.io import loadmat

In [2]:
import numpy as np
from scipy.linalg import expm, logm

In [3]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data_odor = loadmat('/Users/linggeli/neuroscience/data/SuperChris/super_chris_extraction_odor2s.mat')

In [5]:
trial_info = data_odor['trialInfo']
select_odor_A = (trial_info[:, 0] == 1) & (trial_info[:, 1] == 1) & (trial_info[:, 3] == 1)

In [6]:
lfp_odor_A = data_odor['lfpEpoch'][select_odor_A, 2000:4000, 5:10]
lfp_data = lfp_odor_A[:5]

In [7]:
print(lfp_data.shape)

(5, 2000, 5)


In [8]:
cov_series = np.zeros((5, 100, 5, 5))  # time series of covariance matrices
utv_series = np.zeros((5, 100, 15))  # time series of upper triangular vector
for i in range(5):
    for t in range(100):
        lfp_window = lfp_data[i, (t * 10):(t * 10 + 50), :]
        cov = np.cov(lfp_window, rowvar=False)
        cov_series[i, t, :, :] = cov
        utv = logm(cov)[np.triu_indices(5)]
        utv_series[i, t, :] = utv

In [9]:
X = utv_series.reshape((500, 15))
print(X.shape)

(500, 15)


In [10]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
psd_kernels = tfp.positive_semidefinite_kernels


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [11]:
num_points = 100
# Index points should be a collection (100, here) of feature vectors. In this
# example, we're using 1-d vectors, so we just need to reshape the output from
# np.linspace, to give a shape of (100, 1).
index_points = np.expand_dims(np.linspace(-1., 1., num_points), -1)

In [14]:
amplitude = tf.exp(tf.Variable(np.float64(0)), name='amplitude')
length_scale = tf.exp(tf.Variable(np.float64(0)), name='length_scale')
kernel = psd_kernels.ExponentiatedQuadratic(amplitude, length_scale)

observation_noise_variance = tf.exp(
    tf.Variable(np.float64(-5)), name='observation_noise_variance')

In [15]:
gp = tfd.GaussianProcess(
    kernel=kernel,
    index_points=index_points,
    observation_noise_variance=observation_noise_variance)

In [16]:
F = tf.reshape(gp.sample(), [100, 1])

In [17]:
loading = tf.get_variable('loading', [1, 3], np.float64)

In [19]:
Y_hat = tf.matmul(F, loading)

In [22]:
tf.gradients(Y_hat, amplitude)

[<tf.Tensor 'gradients_2/ExponentiatedQuadratic_1/Reshape_1_grad/Reshape:0' shape=() dtype=float64>]

In [21]:
tf.gradients(Y_hat, loading)

[<tf.Tensor 'gradients_1/MatMul_1_grad/MatMul_1:0' shape=(1, 3) dtype=float64>]